Используя схемы переменных направлений и дробных шагов, решить двумерную начально-краевую задачу для дифференциального уравнения параболического типа. В различные моменты времени вычислить погрешность численного решения путем сравнения результатов с приведенным в задании аналитическим решением  $U(x, t)$. 

Исследовать зависимость погрешности от сеточных параметров  $\tau$, $h_x$, $h_y$.

$$ \frac{\partial u}{\partial t} = a \frac{\partial^2 u}{\partial x^2} + b \frac{\partial^2 u}{\partial y^2} + \sin{x} \sin{y} [\cos{\mu t} + (a+b) \sin{\mu t}] $$

$$ u(0, y, t) = 0 \\ u_x(\pi, y, t) = -\sin{y} \sin{\mu t}  \\ u(x, 0, t) = 0 \\  u_y(x, \pi, t) = -\sin{x} \sin{\mu t} \\ u(x, y, 0) = 0

В) $a = 1, b = 2, \mu = 1$

аналитическое решение $$ U(x, y, t) = \sin{x} \sin{y} \sin{\mu t} $$

In [3]:
import math
import typing
from typing import List
import matplotlib.pyplot as plt
import copy
import plotly.graph_objects as go

### Входные условия

In [4]:
x_left = 0
x_right = math.pi
y_left = 0
y_right = math.pi

a = 1
b = 2
mu = 1

In [5]:
def phi_1(y:float, t:float, mu:float=1) -> float:
    return 0

def phi_2(y:float, t:float, mu:float=1) -> float:
    return -math.sin(y) * math.sin(mu * t)


def phi_3(x:float, t:float, mu:float=1) -> float:
    return 0

def phi_4(x:float, t:float, mu:float=1) -> float:
    return -math.sin(x) * math.sin(mu * t)

def psi(x:float, y:float, mu:float=1) -> float:
    return 0

def f(x:float, y:float, t:float, mu:float=1, a:float=1, b:float=2) -> float:
    return math.sin(x) * math.sin(y) * (mu * math.cos(mu * t) + (a + b) * math.sin(mu * t))


def U(x:float, y:float, t:float, mu:float=1) -> float:
    return math.sin(x) * math.sin(y) * math.sin(mu * t)

def real_U(X:List[float], Y:List[float], T:List[float]) -> List[List[List[float]]]:
    U_true = [[[0] * len(X) for _ in range(len(Y))] for __ in range(len(T))]
    for i in range(len(T)):
        for k in range(len(Y)):
            for j in range(len(X)):
                U_true[i][k][j] = U(X[j], Y[k], T[i])
    return U_true

### Вспомогательные функиции

In [6]:
def plot_graphs(new_X:list, new_T:list, found_U:list, U_true:list, s:str='') -> None:
    plt.plot(new_X, U_true[len(new_T) // 4 ],  label='Аналитическое решение')
    plt.plot(new_X, found_U[len(new_T) // 4 ], label=s, linestyle='dashdot')
    plt.plot(new_X, U_true[len(new_T) // 2 ], label='Аналитическое решение')
    plt.plot(new_X, found_U[len(new_T) // 2 ], label=s, linestyle='dashdot')
    plt.plot(new_X, U_true[len(new_T) - 1], label='Аналитическое решение')
    plt.plot(new_X, found_U[len(new_T) - 1],label=s, linestyle='dashdot')
    plt.legend()

In [7]:
'''
I must fix dims HERE
'''

def local_error (U_my:list, U_true:list) -> float:
    return sum([(a - b) ** 2 for a, b in zip(U_my, U_true)]) / len(U_my)

def get_error_array_with_h(N:list, x_left:float, x_right:float, y_left:float, y_right:float, a:float, b:float, c:float, d:float, 
                           find_u:typing.Callable, eps:float=0.1, omega:float=-10)  -> (list, list): # H, error
    

    H_x = [x_right/(n - 1) for n in N]
    H_y = [y_right/(n - 1) for n in N]
    ERROR = []
    for n in N:
        if omega == -10:
            XX, YY, UU = find_u(x_left, x_right, y_left, y_right, a, b, c, d, eps=eps, n_x=n, n_y=n)
        else:
            XX, YY, UU = find_u(x_left, x_right, y_left, y_right, a, b, c, d=0, eps=eps, n_x=n, n_y=n, omega=omega)
        U_true = real_U(XX, YY)
        t = len(YY) // 2
        ERROR.append(local_error(UU[t], U_true[t]))
    
    return H_x, H_y ,ERROR

In [8]:
def h_error_plot(H:list, ERROR:list, s:str=' x') -> None:
    plt.plot(H, ERROR[::-1])
    plt.xlabel("h" + s)
    plt.ylabel("error")
    plt.show()

In [9]:
def frange(start:float, stop:float, step:float) -> float:
    while start < stop:
        yield start
        start += step

In [10]:
def get_y(y0:float, y_end:float, h:float) -> List[float]:
    return [i for i in frange(y0, y_end+h, h)]

def get_x(x_0:float, x_l:float, h:float) -> List[float]:
    return [i for i in frange(x_0, x_l+h, h)]

def get_t(t_0:float, t_end:float, h:float) -> List[float]:
    return [i for i in frange(t_0, t_end+h, h)]

In [11]:
def solve_PQ(A0:list, A1:list, A2:list, B:list) -> list:
    P = [-A2[0] / A1[0]]
    Q = [B[0] / A1[0]]
    for i in range(1, len(B)):

        P.append(-A2[i] / (A1[i] + A0[i] * P[i - 1]))
        Q.append((B[i] - A0[i] * Q[i - 1]) / (A1[i] + A0[i] * P[i - 1]))

    res = [Q[-1]]

    for i in range(len(B) - 2, -1, -1):
        res.append(P[i] * res[-1] + Q[i])

    return res[::-1]

In [12]:
def error(U_prev:List[List[float]], U_values:List[List[float]]) -> float:
    eps = 0
    for i in range(len(U_values)):
        for j in range(len(U_values[0])):
            eps = max(eps, abs(U_prev[i][j] - U_values[i][j]))
    return eps


In [21]:
def border_conds(U_values:List[List[float]], X:List[float], Y:List[float], T:List[float], mu:float, hx:float, hy:float) -> None:
    # for k in range(len(T)):
    #     for i in range(len(Y)):
    #         U_values[k][i][0] = phi_1(Y[i], T[k], mu=mu)
    #         U_values[k][i][-1] = phi_2(Y[i], T[k], mu=mu) * hy +  U_values[k][i][-2] 
    #     for j in range(len(X)):
    #         U_values[k][0][j] = phi_3(X[j], T[k], mu=mu) 
    #         U_values[k][-1][j] = phi_4(X[j], T[k], mu=mu) * hx + U_values[k][-2][i]

    for i in range(len(Y)):
        for i in range(len(X)):
            U_values[0][i][j] = psi(X[j], Y[i], mu=mu)
    for k in range(len(T)):
        for i in range(len(X)):
            U_values[k][i][0] = phi_3(X[i], T[k], mu=mu)
            U_values[k][i][-1] = phi_4(X[i], T[k], mu=mu) * 2 * hy + U_values[k][i][-2]
        for j in range(len(Y)):
            U_values[k][0][j] = phi_1(Y[j], T[k], mu=mu)
            U_values[k][-1][j] = phi_2(Y[j], T[k], mu=mu) * 2 * hx + U_values[k][-2][j]

In [22]:
def initial_T(U_values:List[List[float]], X:List[float], Y:List[float], mu:float) -> None:
    for i in range(len(Y)):
        for i in range(len(X)):
            U_values[0][i][j] = psi(X[j], Y[i], mu=mu)

### Метод переменных направлений

$$ \frac{u^{k + \frac{1}{2}}_{i,j} - u^{k}_{i,j}}{\frac{\tau}{2}} = \frac{a}{h_1^2} \left (  u^{k + \frac{1}{2}}_{i+1, j} - 2  u^{k + \frac{1}{2}}_{i, j} +  u^{k + \frac{1}{2}}_{i-1, j}\right )  + \frac{b}{h_2^2} \left (  u^{k}_{i, j+1} - 2  u^{k}_{i, j} +  u^{k}_{i, j-1}\right ) + f^{k + \frac{1}{2}}_{i, j}$$
идём по j = $1 .. y_l - h_y$ по x неявно, по y явно


$$ \frac{u^{k + 1}_{i,j} - u^{k + \frac{1}{2}}_{i,j}}{\frac{\tau}{2}} = \frac{a}{h_1^2} \left (  u^{k + \frac{1}{2}}_{i+1, j} - 2  u^{k + \frac{1}{2}}_{i, j} +  u^{k + \frac{1}{2}}_{i-1, j}\right )  + \frac{b}{h_2^2} \left (  u^{k + 1}_{i, j+1} - 2  u^{k + 1}_{i, j} +  u^{k + 1}_{i, j-1}\right ) + f^{k + \frac{1}{2}}_{i, j}$$

идём по i = $1 .. x_l - h_x$ по x явно, по y неявно

In [14]:
def changing_directions(x_left:float, x_right:float, y_left:float, y_right:float, a:float, b:float, mu:float, 
                        n_x:float=10, n_y:float=10, t_end:float=1) -> (List[float], List[float], List[List[float]], List[List[float]]):
    t_right = t_end
    hx = x_right / (n_x - 1)
    hy = y_right / (n_y - 1)
    tau = (hx**2) / 2 / a**2 


    X = get_x(x_left, x_right, hx)
    Y = get_y(y_left, y_right, hy)
    T = get_t(0, t_right, tau)


    U_values = [[[0] * len(Y) for _ in range(len(X))] for __ in range(len(T))]
    sigma_x = (a * tau) / hx**2
    sigma_y = (b * tau) / hy**2
    #border_conds(U_values, X, Y, T, mu, hx, hy)
    #initial_T(U_values)
    A0 = [0 for _ in range(len(X))]
    A1 = [0 for _ in range(len(X))]
    A2 = [0 for _ in range(len(X))]
    B = [0 for _ in range(len(X))]
    for k in range(1, len(T)):
        U_part = [[0 for _ in range(len(X))] for _ in range(len(Y))]
        t_part = T[k-1] + tau/2
        for i in range(1, len(X) - 1):
            #A0[0] = 0
            #тут не уверен, надо бует пересмотреть
            A1[0] = 1
            A2[0] = 0
            B[0] = 0#phi_3(X[j], t_part, mu=mu) 
            for j in range(1, len(Y) - 1):
                A0[i] = - sigma_x
                A1[i] = 1 + 2 * sigma_x
                A2[i] = - sigma_x
                B[i] = f(X[i], Y[j], t_part, mu=mu) * tau / 2 + sigma_y ....# дописать еще штук
            A0[-1] = -1
            A1[-1] =  1
            #A2[-1] = 0 
            B[-1] = 0#phi_4(Y[j], t_part, mu=mu) * 2 * hy + U_values[k][-2][-1] ### тут надо подумать что брать U_values
            tmp_res = solve_PQ(A0, A1, A2, B)

            
            U_part = tmp_res
            # добавляем начальные значения и граничные
        t_part += tau/2
        for j in range(1, len(Y) - 1):
            A1[0] = 1
            A2[0] = 0
            B[0] = 0#phi_1(Y[j], t_part, mu=mu)
            for i in range(1, len(X) - 1):
                A0[i] = - sigma_y
                A1[i] = 1 + 2 * sigma_y
                A2[i] = - sigma_y
                B[i] = f(X[i], Y[j], t_part - tau/2, mu=mu) * tau / 2 + sigma_x ... # дописать еще штук U_part
            A0[-1] = -1
            A1[-1] =  1
            #A2[-1] = 0 
            B[-1] = 0#phi_2(X[j], t_part, mu=mu) * 2 * hx + U_values[k][-2][-1] # аналогично предидущему варианту с шраниццами
            tmp_res = solve_PQ(A0, A1, A2, B) # езультат, заносим в U_values

            
    


    
    return X, Y, T, U_values

In [15]:
X, Y, T, UU = changing_directions(x_left, x_right, y_left, y_right, a, b, mu, n_x=10, n_y=10, t_end=1)
U_true = real_U(X, Y, T)

In [18]:
fig = go.Figure(data=[go.Surface(z=UU[-1], x=X, y=Y)])
fig.show()


In [19]:
UU[-1]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, -0.20539891013243483],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, -0.3860236803378339],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, -0.520088297591727],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, -0.5914225904702688],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, -0.5914225904702688],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, -0.5200882975917271],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, -0.3860236803378341],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, -0.20539891013243497],
 [0.0,
  -0.20539891013243483,
  -0.3860236803378339,
  -0.520088297591727,
  -0.5914225904702688,
  -0.5914225904702688,
  -0.5200882975917271,
  -0.3860236803378341,
  -0.20539891013243497,
  -0.20539891013243505]]

In [20]:
U_true[-1]

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0,
  0.10062652161785311,
  0.1891159996393005,
  0.2547952970493493,
  0.2897425212571536,
  0.2897425212571536,
  0.25479529704934933,
  0.18911599963930056,
  0.10062652161785315,
  3.603061106587963e-17],
 [0.0,
  0.1891159996393005,
  0.35542181866720235,
  0.4788585208964541,
  0.5445378183065029,
  0.5445378183065029,
  0.4788585208964542,
  0.35542181866720246,
  0.1891159996393006,
  6.771539868202835e-17],
 [0.0,
  0.2547952970493493,
  0.4788585208964541,
  0.645164339924356,
  0.7336538179458034,
  0.7336538179458034,
  0.6451643399243561,
  0.47885852089645425,
  0.25479529704934945,
  9.123270984427612e-17],
 [0.0,
  0.2897425212571536,
  0.5445378183065029,
  0.7336538179458034,
  0.8342803395636565,
  0.8342803395636565,
  0.7336538179458035,
  0.544537818306503,
  0.2897425212571538,
  1.0374600974790797e-16],
 [0.0,
  0.2897425212571536,
  0.5445378183065029,
  0.7336538179458034,
  0.8342803395636565,
  0.83428

In [17]:
fig = go.Figure(data=[go.Surface(z=U_true[-1], x=X, y=Y)])
fig.show()

### Метод дробных шагов

Здесь слой $k + \frac{1}{2}$ становится "полностью вирутальныйм"

$$ \frac{u^{k + \frac{1}{2}}_{i,j} - u^{k}_{i,j}}{\tau} = \frac{a}{h_1^2} \left (  u^{k + \frac{1}{2}}_{i+1, j} - 2  u^{k + \frac{1}{2}}_{i, j} +  u^{k + \frac{1}{2}}_{i-1, j}\right ) + f^{k}_{i, j}$$

идём по j = $1 .. y_l - h_y$

$$ \frac{u^{k + 1}_{i,j} - u^{k + \frac{1}{2}}_{i,j}}{\tau} = \frac{b}{h_2^2} \left (  u^{k + 1}_{i, j+1} - 2  u^{k + 1}_{i, j} +  u^{k + 1}_{i, j-1}\right ) + f^{k + 1}_{i, j}$$

идём по i = $1 .. x_l - h_x$ по x явно, по y неявно

In [ ]:
def partial_steps():
    return X, Y, T, U_values